In [1]:
import numpy as np
import itertools
import pandas as pd
import json
import sys
import seaborn as sb
import matplotlib.pyplot as plt
# For regular expressions
import re
# For handling string
import string
import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Read just the ingredient column from the reduced veg recipes


In [5]:
#Read just the csv for ingredients column
df = pd.read_csv('/content/drive/MyDrive/Project 298/Final Project Files/EDA Notebooks/recipes_reduced_veg_recipes.csv')
df.head()

,title,ingredients,instructions
0,dilly macaroni salad recipe,"['1 c. elbow macaroni', '1 c. cubed American c...",['Cook macaroni according to package direction...
1,gazpacho,"['8 tomatoes, quartered', 'Kosher salt', '1 re...",['Add the tomatoes to a food processor with a ...
2,crunchy onion potato bake,"['2 12 cups milk', '1 12 cups water', '14 cup ...","['Preheat oven to 350 degrees Fahrenheit.', 'S..."
3,cool 'n easy creamy watermelon pie,"['1 (3 ounce) package watermelon gelatin', '14...","['Dissolve Jello in boiling water.', 'Allow to..."
4,strawberry rhubarb dump cake,"['6 -8 cups fresh rhubarb, or', '6 -8 cups fro...",['Put ingredients in a buttered 9 x 12 x 2-inc...


In [6]:
df_sample_2= df.head(100000)

Ingredients for a recepie needs to be read separately to extract quantity, measure and the ingredient itself for a given recipe. Need to keeo the index to recognize later on recipe index-ingredient relationship.

Splitting each ingredient/recipe into three different columns. This code takes care that the column is not split by  ',' in the sentence between single quotes

In [7]:
l2=df_sample_2.assign(ingredients=df_sample_2.ingredients.str.split("'(?=,)")).explode('ingredients')
l2

,title,ingredients,instructions
0,dilly macaroni salad recipe,['1 c. elbow macaroni,['Cook macaroni according to package direction...
0,dilly macaroni salad recipe,", '1 c. cubed American cheese (4 ounce.)",['Cook macaroni according to package direction...
0,dilly macaroni salad recipe,", '1/2 c. sliced celery",['Cook macaroni according to package direction...
0,dilly macaroni salad recipe,", '1/2 c. minced green pepper",['Cook macaroni according to package direction...
0,dilly macaroni salad recipe,", '3 tbsp. minced pimento",['Cook macaroni according to package direction...
...,...,...,...
99998,oatmeal bread (abm),", '1 egg, lightly beaten",['Combine first 5 ingredients in a bowl; stir ...
99998,oatmeal bread (abm),", '1 (1/4 ounce) package dry yeast']",['Combine first 5 ingredients in a bowl; stir ...
99999,manx herrings and priddhas (potatoes),"['2 medium potatoes, parboiled and thickly sliced","['Preheat oven to 425F.', 'Place potatoes in a..."
99999,manx herrings and priddhas (potatoes),", '2 whole herrings, cleaned","['Preheat oven to 425F.', 'Place potatoes in a..."


In [8]:
ing_final=pd.DataFrame(l2['ingredients'])
ing_final.head()

,ingredients
0,['1 c. elbow macaroni
0,", '1 c. cubed American cheese (4 ounce.)"
0,", '1/2 c. sliced celery"
0,", '1/2 c. minced green pepper"
0,", '3 tbsp. minced pimento"


Removing the unnecessary quotes

In [9]:
ing_final = ing_final['ingredients'].str.replace('[^\w\s]', '')
# ing_final = ing_final.str.replace('[^\w\s]', '')


<ipython-input-9-c6d49c1a0744>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  ing_final = ing_final['ingredients'].str.replace('[^\w\s]', '')


Removing extra space after cleaning

In [10]:
ing_final = ing_final.replace(r"^ +| +$", r"", regex=True)
ing_final.head()

0                   1 c elbow macaroni
0    1 c cubed American cheese 4 ounce
0                   12 c sliced celery
0             12 c minced green pepper
0                3 tbsp minced pimento
Name: ingredients, dtype: object

In [11]:
replace_values = {'12' : '1/2', '34' : '3/4', '14' : '1/4' , '13' : '1/3'}                                                                                          
r_ingredients = ing_final.replace(replace_values, regex=True)                                                                                             
r_ingredients.head()

0                   1 c elbow macaroni
0    1 c cubed American cheese 4 ounce
0                  1/2 c sliced celery
0            1/2 c minced green pepper
0                3 tbsp minced pimento
Name: ingredients, dtype: object

In [15]:
r_ingredients= pd.DataFrame(r_ingredients) #No Need , can be skipped

In [ ]:
#primary cleaning
#convert vulgar fractions 
import unicodedata
unicodedata.numeric(u'⅕')
unicodedata.name(u'⅕')

#convert vulgar fractions
for ix, row in r_ingredients.iterrows():
    for char in row['ingredients']:
        if unicodedata.name(char).startswith('VULGAR FRACTION'):  
            normalized = unicodedata.normalize('NFKC', char)
            r_ingredients.iloc[ix, 1] = r_ingredients.iloc[ix, 1].replace(char, normalized)

In [12]:
r_ingredients.head(30)

0                             1 c elbow macaroni
0              1 c cubed American cheese 4 ounce
0                            1/2 c sliced celery
0                      1/2 c minced green pepper
0                          3 tbsp minced pimento
0    1/2 c mayonnaise or possibly salad dressing
0                                 1 tbsp vinegar
0                              3/4 teaspoon salt
0                     1/2 teaspoon dry dill weed
1                           8 tomatoes quartered
1                                    Kosher salt
1                1 red onion cut into small dice
1        1 green bell pepper cut into small dice
1          1 red bell pepper cut into small dice
1       1 yellow bell pepper cut into small dice
1               1/2 cucumber cut into small dice
1            Extravirgin olive oil for drizzling
1            3 leaves fresh basil finely chopped
2                                2 1/2 cups milk
2                               1 1/2 cups water
2                   

Splitting the column into "Unit", "Quantity" and "ingredient "

In [ ]:
# df_ing_split= r_ingredients.str.split(" ", n=2,expand= True)

In [ ]:
# df_ing_split.head(40)

,0,1,2
0,6,ounces,penne
0,2,cups,Beechers Flagship Cheese Sauce recipe follows
0,1,ounce,Cheddar grated 1/4 cup
0,1,ounce,Gruyere cheese grated 1/4 cup
0,1/4,to,1/2 teaspoon chipotle chili powder see Note
0,1/4,cup,1/2 stick unsalted butter
0,1/3,cup,allpurpose flour
0,3,cups,milk
0,1/4,ounces,semihard cheese page 23 grated about 3 1/2 cups
0,2,ounces,semisoft cheese page 23 grated 1/2 cup


Rename the columns 

In [ ]:
# df_ing_split.rename(columns={df_ing_split.columns[0]:'quantity',df_ing_split.columns[1]:'unit',df_ing_split.columns[2]:'ingredients'},inplace=True)


In [ ]:
# df_ing_split.head()

,quantity,unit,ingredients
0,6,ounces,penne
0,2,cups,Beechers Flagship Cheese Sauce recipe follows
0,1,ounce,Cheddar grated 1/4 cup
0,1,ounce,Gruyere cheese grated 1/4 cup
0,1/4,to,1/2 teaspoon chipotle chili powder see Note


In [ ]:
# from itertools import chain
# def escape_re_string(text):
#     text = text.replace('.', '\.')
#     return re.sub(r'\s+', ' ', text)

# UNITS = {"cup": ["cups", "cup", "c.", "c"], "fluid_ounce": ["fl. oz.", "fl oz", "fluid ounce", "fluid ounces"],
#          "gallon": ["gal", "gal.", "gallon", "gallons"], "ounce": ["oz", "oz.", "ounce", "ounces"],
#          "pint": ["pt", "pt.", "pint", "pints"], "pound": ["lb", "lb.", "pound", "pounds"],
#          "quart": ["qt", "qt.", "qts", "qts.", "quart", "quarts"],
#          "tablespoon": ["tbsp.", "tbsp", "T", "T.", "tablespoon", "tablespoons", "tbs.", "tbs"],
#          "teaspoon": ["tsp.", "tsp", "t", "t.", "teaspoon", "teaspoons"],
#          "gram": ["g", "g.", "gr", "gr.", "gram", "grams"], "kilogram": ["kg", "kg.", "kilogram", "kilograms"],
#          "liter": ["l", "l.", "liter", "liters"], "milligram": ["mg", "mg.", "milligram", "milligrams"],
#          "milliliter": ["ml", "ml.", "milliliter", "milliliters"], "pinch": ["pinch", "pinches"],
#          "dash": ["dash", "dashes"], "touch": ["touch", "touches"], "handful": ["handful", "handfuls"],
#          "stick": ["stick", "sticks"], "clove": ["cloves", "clove"], "can": ["cans", "can"], "large": ["large"],
#          "small": ["small"], "scoop": ["scoop", "scoops"], "filets": ["filet", "filets"], "sprig": ["sprigs", "sprig"]}

# # a = df_ing_split.unit(chain.from_iterable(UNITS.values()))
# # a.sort(key=lambda x: len(x), reverse=True)
# # a = map(escape_re_string, a)
# df_ing_split['unit'] = df_ing_split['unit'] .replace(UNITS)  

Separating just the ingredient from ingredient column 

In [ ]:
df_ing_work=pd.DataFrame(df_ing_split['ingredients'])

In [ ]:
df_ing_work.head()

,ingredients
0,penne
0,Beechers Flagship Cheese Sauce recipe follows
0,Cheddar grated 1/4 cup
0,Gruyere cheese grated 1/4 cup
0,1/2 teaspoon chipotle chili powder see Note


Using NER just to extract the ingredients

In [13]:
# #NER to clean data
# !pip install --upgrade spacy
# import spacy
#load the existing small model from spacy
base_model = spacy.load('en_core_web_sm')

In [16]:
# https://github.com/vishwapardeshi/NL_Parser_using_Spacy/blob/master/notebooks/Recipe_Ingredient_Parser.ipynb
measurements = re.compile(r'(c.|tsp|tbsp|lb|lbs|bowl|bulb|cube|clove|cup|drop|ounce|oz|pinch|pound|teaspoon|tablespoon)s?')
r_ingredients['ingredients']= r_ingredients['ingredients'].astype(str)
extracted = []

for ix, row in r_ingredients.iterrows():
    print('\r', "Extracting ingredient for row", ix, end='')
    tokens = base_model(row['ingredients'])
    extract = ''
    for token in tokens:
        if (token.dep_ in ['nsubj', 'ROOT']) and (token.pos_ in ['NOUN', 'PROPN']) and (not measurements.match(token.text)):
        #explore children
            for child in token.children:
                if (not measurements.match(child.text)) and (child.dep_ in ['amod', 'compound']):
                    extract += child.text + ' '
            extract += token.text + ' '
    extracted.append(extract)

 Extracting ingredient for row 99999

In [17]:
r_ingredients['cleaned ingredients'] = extracted
r_ingredients.head(50)

,ingredients,cleaned ingredients
0,1 c elbow macaroni,elbow
0,1 c cubed American cheese 4 ounce,
0,1/2 c sliced celery,
0,1/2 c minced green pepper,minced green pepper
0,3 tbsp minced pimento,
0,1/2 c mayonnaise or possibly salad dressing,c mayonnaise
0,1 tbsp vinegar,vinegar
0,3/4 teaspoon salt,salt
0,1/2 teaspoon dry dill weed,dry dill weed
1,8 tomatoes quartered,tomatoes


In [18]:
r_ingredients.to_csv('extracted_ingredients_100000.csv', index=False)